In [1]:
import torch, math, copy
import numpy as np
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# Replace with path to MNIST on your machine
train_dataset = datasets.MNIST("/Users/lucastucker/REU-2023/archive/", train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

test_dataset = datasets.MNIST("/Users/lucastucker/REU-2023/archive/", train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

"""
targets = torch.zeros(10000, 1)
for i, (data, target) in enumerate(test_loader):
    targets[i] = target
    # print(type(targets[i]))
targets = DataLoader(TensorDataset(targets), batch_size=256, shuffle=False)
# print(len(targets))
for target in targets:
    print(target[0].view(1, -1).shape)
"""

'\ntargets = torch.zeros(10000, 1)\nfor i, (data, target) in enumerate(test_loader):\n    targets[i] = target\n    # print(type(targets[i]))\ntargets = DataLoader(TensorDataset(targets), batch_size=256, shuffle=False)\n# print(len(targets))\nfor target in targets:\n    print(target[0].view(1, -1).shape)\n'

In [3]:
def train(epochs, model, criterion, optimizer, train_loader, test_loader, reduced_dim, t_nearest, batch_size):
    for epoch in range(epochs):
        train_err = train_epoch(model, criterion, optimizer, train_loader, reduced_dim, t_nearest, batch_size)
        test_err = test(model, test_loader, reduced_dim, t_nearest, batch_size)
        print('Epoch {:03d}/{:03d}, Train Error {:.2f}% || Test Error {:.2f}%'.format(epoch, epochs, train_err*100, test_err*100))
    return train_err, test_err

In [4]:
def train_epoch(model, criterion, optimizer, loader, reduced_dim, t_nearest, batch_size):
    total_correct = 0.
    total_samples = 0.

    data_matrix = np.zeros((60000, 784))
    targets = torch.zeros((60000, 1))

    for batch_idx, (data, target) in enumerate(loader):
        data_matrix[batch_idx] = data.view(1, 784)    
        targets[batch_idx] = target    

    data_matrix = le_on_loader(data_matrix, reduced_dim ** 2, t_nearest, sigma=0.3)
    # print(f"data matrix looks like {data_matrix}")
    reduced_tensor = torch.Tensor(data_matrix).view(-1, 1, reduced_dim, reduced_dim)

    targets = DataLoader(TensorDataset(torch.Tensor(targets)), batch_size=batch_size, shuffle=False)
    dataloader = DataLoader(TensorDataset(reduced_tensor), batch_size=batch_size, shuffle=False)

    # You can now iterate over dataloader for mini-batch processing
    for data, target in zip(dataloader, targets):
        # NOTE: Uncomment the code below if you are using a GPU
        # if torch.cuda.is_available():
        #    data, target = data.cuda(), target.cuda()
        print(f"data[0] is of shape {data[0].shape}")
        print(f"target[0] is of shape {target[0].shape}")
        print(f"target[0].view(-1) has shape {target[0].view(-1).shape}")

        output = model(data[0])

        # long vs float issue
        loss = F.cross_entropy(output, target[0].view(-1).long())
        preds = output.argmax(dim=1, keepdim=True)
        total_correct += preds.eq(target[0].long().view_as(preds)).sum().item() # compare preds to target
        total_samples += torch.numel(preds) # numel short for number of elements
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return 1 - total_correct/total_samples

In [5]:
def test(model, loader, reduced_dim, t_nearest, batch_size):
    total_correct = 0.
    total_samples = 0.
    model.eval()

    data_matrix = np.zeros((10000, 784))
    targets = np.zeros((10000, 1))

    for batch_idx, (data, target) in enumerate(loader):
        data_matrix[batch_idx] = data.view(1, 784)    
        targets[batch_idx] = target    

    data_matrix = le_on_loader(data_matrix, reduced_dim ** 2, t_nearest, 0.3)
    reduced_tensor = torch.Tensor(data_matrix).view(-1, 1, reduced_dim, reduced_dim)

    # Create a DataLoader
    targets = DataLoader(TensorDataset(torch.Tensor(targets)), batch_size=batch_size, shuffle=False)
    dataloader = DataLoader(TensorDataset(reduced_tensor), batch_size=batch_size, shuffle=False)

    with torch.no_grad():
        for data, target in zip(dataloader, targets):
            # NOTE: Uncomment the code below if you are using a GPU
            # if torch.cuda.is_available():
            #    data, target = data.cuda(), target.cuda()

            output = model(data[0])
            preds = output.argmax(dim=1, keepdim=True)
            total_correct += preds.eq(target[0].long().view_as(preds)).sum().item()
            total_samples += preds.numel()

    return 1 - total_correct/total_samples

In [6]:
%run /Users/lucastucker/REU-2023/laplacian_eigenmaps_functions.ipynb

In [7]:
def le_on_loader(X, n_components, t_neighbors, sigma):
    return laplacian_eigenmaps(X, n_components, t_neighbors, sigma)

In [8]:
class CNNeluBN(nn.Module):
    def __init__(self, reduced_dim):
        super(CNNeluBN, self).__init__()

        # write code here to instantiate layers
        # for example, self.conv = nn.Conv2d(1, 4, 3, 1, 1)
        # creates a conv layer with 1 input channel, 4 output
        # channels, a 3x3 kernel, and stride=padding=1
        self.layers = nn.ModuleList()

        self.layers.append(nn.Conv2d(1, 4, 3, 1, 1)) # 1 to 4 channels on 1 x red x red input
        self.layers.append(nn.BatchNorm2d(4))
        self.layers.append(nn.ELU())
        self.layers.append(nn.AvgPool2d(2, 2)) # Now size is 4 x red // 2 x red // 2
        self.layers.append(nn.Conv2d(4, 8, 3, 1, 1)) # 4 to 8 channels
        self.layers.append(nn.BatchNorm2d(8))
        self.layers.append(nn.ELU())
        self.layers.append(nn.AvgPool2d(2, 2)) # Now size is 8 x red // 2 // 2 x red // 2 // 2
        new_dim = (reduced_dim // 2) // 2
        self.layers.append(nn.Linear(8 * new_dim * new_dim, 10))

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                sigma = 1 / (math.sqrt(9 * m.out_channels)) # 9 is k^2
                m.weight.data.normal_(0, sigma)
                m.bias.data.zero_()

    def forward(self, input):
      # print(input.shape)
      u = self.layers[0](input)
      for layer in self.layers[1:-1]:
         u = layer(u)
      num_batches = u.size()[0]
      u = u.view(num_batches, -1)
      return self.layers[-1](u)

In [9]:
criterion = torch.nn.CrossEntropyLoss() # TODO (implement in nn) 

In [10]:
lr = 0.05
reduced_dim = 20
t_nearest = 50
batch_size = 512
print(f"Training ELU CNN + BN with representation dimension {reduced_dim} x {reduced_dim}")
model = CNNeluBN(reduced_dim) # .cuda() 
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
print(f"train_loader")
train_errs, test_errs = train(3, model, criterion, optimizer, train_loader, test_loader, reduced_dim, t_nearest, batch_size)

Training ELU CNN + BN with representation dimension 20 x 20
train_loader
called laplacian_eigenmaps!
got nearest neighbors, distances, indices!
Made W with shape (60000, 60000)


: 